In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import joblib
from rembg import remove


## 영숙 데이터 전처리 - 1

In [83]:
data = np.load('youngsook.npz')
array1 = data['data1']
array1.shape

(150, 160, 340, 4)

### 4차원으로 배열변환

In [84]:
array1=array1.reshape(150,160, 340, 4)
array1.shape

(150, 160, 340, 4)

### 그레이스케일 후 이미지를 1차원으로 벡터로 변환

In [85]:
a=[]
for i in array1:
    gray = cv2.cvtColor(i, cv2.COLOR_RGBA2GRAY)
    gray=gray.reshape(-1)
    a.append(gray)

In [109]:
asdf=pd.DataFrame(a)

In [110]:
asdf

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
139,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
140,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
141,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [111]:
image_data = asdf.iloc[0].values

In [112]:
image_data.shape

(54400,)

### 이미지 증폭

In [113]:
from albumentations import (
    Compose,
    HorizontalFlip,
    VerticalFlip,
    RandomRotate90,
    Rotate,
    GaussNoise,
    RandomScale,
    RandomBrightnessContrast
)

In [114]:
# Albumentations에서 제공하는 여러 가지 증강 기법 중 일부 선택 (예시)

augmenter = Compose([
    HorizontalFlip(p=0.5),
    Rotate(limit=45, p=0.5),  # -45도에서 45도 사이의 임의의 각도로 회전
    GaussNoise(var_limit=(10.0, 50.0), p=0.5),  # 가우시안 노이즈 추가
    RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),  # 밝기와 대비 조절
])

augmented_rows = []

for row_index in range(len(asdf)):
    image_data = asdf.iloc[row_index].values
    image_data_reshaped = image_data.reshape(160, 340)

    # 원본 이미지 저장
    augmented_rows.append(image_data_reshaped.flatten())

    for _ in range(7):  # 원본 포함하여 5배로 증가시키기 위해 4번 반복
        # 증강된 이미지 생성 및 저장 
        augmented_image = augmenter(image=image_data_reshaped)['image']
        augmented_rows.append(augmented_image.flatten())

# DataFrame에 추가하기 위해 numpy array로 변환
augmented_array = np.array(augmented_rows)

# 새 DataFrame 생성 (원본 DataFrame의 컬럼 이름 사용)
df_augmented = pd.DataFrame(data=augmented_array, columns=asdf.columns)

In [115]:
youngsookDF=df_augmented
youngsookDF

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,2,0,0,0,...,0,0,3,0,1,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,4,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139,0,0,0,0,0,0,0,0,0,0,...,2,2,1,1,1,1,0,1,3,4
1140,0,0,4,0,0,2,1,0,2,5,...,1,2,0,0,5,3,0,0,0,0
1141,0,0,0,7,0,3,10,0,0,4,...,0,1,3,2,6,0,8,0,11,6
1142,0,0,0,0,0,0,0,2,1,8,...,6,0,1,0,0,0,0,2,0,12


### 이미지 라벨링

In [116]:
youngsookDF['woman_num']=1

## 정숙 데이터 전처리 2

In [117]:
data = np.load('jungsook.npz')
array2 = data['data1']
array2.shape

(143, 160, 340, 4)

### 4차원으로 배열변환

In [118]:
array2=array2.reshape(143,160, 340, 4)
array2.shape

(143, 160, 340, 4)

### 그레이스케일 후 이미지를 1차원으로 벡터로 변환

In [119]:
a=[]
for i in array2:
    gray = cv2.cvtColor(i, cv2.COLOR_RGBA2GRAY)
    gray=gray.reshape(-1)
    a.append(gray)

In [123]:
asdf=pd.DataFrame(a)
asdf

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
139,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
140,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
141,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [124]:
image_data = asdf.iloc[0].values
image_data.shape

(54400,)

### 이미지 증폭

In [125]:
# Albumentations에서 제공하는 여러 가지 증강 기법 중 일부 선택 (예시)

augmenter = Compose([
    HorizontalFlip(p=0.5),
    Rotate(limit=45, p=0.5),  # -45도에서 45도 사이의 임의의 각도로 회전
    GaussNoise(var_limit=(10.0, 50.0), p=0.5),  # 가우시안 노이즈 추가
    RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),  # 밝기와 대비 조절
])

augmented_rows = []

for row_index in range(len(asdf)):
    image_data = asdf.iloc[row_index].values
    image_data_reshaped = image_data.reshape(160, 340)

    # 원본 이미지 저장
    augmented_rows.append(image_data_reshaped.flatten())

    for _ in range(7):  # 원본 포함하여 5배로 증가시키기 위해 4번 반복
        # 증강된 이미지 생성 및 저장 
        augmented_image = augmenter(image=image_data_reshaped)['image']
        augmented_rows.append(augmented_image.flatten())

# DataFrame에 추가하기 위해 numpy array로 변환
augmented_array = np.array(augmented_rows)

# 새 DataFrame 생성 (원본 DataFrame의 컬럼 이름 사용)
df_augmented = pd.DataFrame(data=augmented_array, columns=asdf.columns)

In [126]:
jungsookDF=df_augmented
jungsookDF

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6,3,0,3,0,3,0,4,1,0,...,0,3,0,0,1,0,0,0,0,0
2,0,0,5,0,0,2,4,1,0,0,...,6,2,0,0,0,3,6,0,0,0
3,24,24,31,24,24,30,24,30,27,30,...,25,24,24,27,24,26,24,24,24,27
4,4,3,0,0,0,0,4,0,3,0,...,0,0,5,2,0,0,0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139,1,3,0,0,1,1,6,3,0,6,...,0,5,3,0,0,0,0,0,7,9
1140,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1141,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1142,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 이미지 라벨링

In [127]:
jungsookDF['woman_num']=2

## 순자 데이터 전처리3

In [128]:
data = np.load('soonja.npz')
array3 = data['data1']
array3.shape

(183, 160, 340, 4)

### 4차원으로 배열변환

In [129]:
array3=array3.reshape(183,160, 340, 4)
array3.shape

(183, 160, 340, 4)

### 그레이스케일 후 이미지를 1차원으로 벡터로 변환

In [130]:
a=[]
for i in array3:
    gray = cv2.cvtColor(i, cv2.COLOR_RGBA2GRAY)
    gray=gray.reshape(-1)
    a.append(gray)

In [131]:
asdf=pd.DataFrame(a)
asdf

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,58,60,63,66,68,69,73,74,72,65
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
180,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
181,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [132]:
image_data = asdf.iloc[0].values
image_data.shape

(54400,)

### 이미지 증폭

In [133]:
# Albumentations에서 제공하는 여러 가지 증강 기법 중 일부 선택 (예시)

augmenter = Compose([
    HorizontalFlip(p=0.5),
    Rotate(limit=45, p=0.5),  # -45도에서 45도 사이의 임의의 각도로 회전
    GaussNoise(var_limit=(10.0, 50.0), p=0.5),  # 가우시안 노이즈 추가
    RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),  # 밝기와 대비 조절
])

augmented_rows = []

for row_index in range(len(asdf)):
    image_data = asdf.iloc[row_index].values
    image_data_reshaped = image_data.reshape(160, 340)

    # 원본 이미지 저장
    augmented_rows.append(image_data_reshaped.flatten())

    for _ in range(7):  # 원본 포함하여 5배로 증가시키기 위해 4번 반복
        # 증강된 이미지 생성 및 저장 
        augmented_image = augmenter(image=image_data_reshaped)['image']
        augmented_rows.append(augmented_image.flatten())

# DataFrame에 추가하기 위해 numpy array로 변환
augmented_array = np.array(augmented_rows)

# 새 DataFrame 생성 (원본 DataFrame의 컬럼 이름 사용)
df_augmented = pd.DataFrame(data=augmented_array, columns=asdf.columns)

In [134]:
soonjaDF=df_augmented
soonjaDF

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,46,46,46,46,46,46,46,46,46,46,...,46,46,46,46,46,46,46,46,46,46
2,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
3,0,0,0,0,0,0,0,0,0,0,...,154,150,145,148,131,121,109,107,104,70
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1460,0,12,0,0,3,6,0,0,2,13,...,0,2,0,3,0,0,0,0,8,3
1461,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1462,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 이미지 라벨링

In [135]:
soonjaDF['woman_num']=3

## 영자 데이터 전처리4

In [136]:
data = np.load('youngja.npz')
array4 = data['data1']
array4.shape

(173, 160, 340, 4)

### 4차원으로 배열변환

In [137]:
array4=array4.reshape(173,160, 340, 4)
array4.shape

(173, 160, 340, 4)

### 그레이스케일 후 이미지를 1차원으로 벡터로 변환

In [138]:
a=[]
for i in array4:
    gray = cv2.cvtColor(i, cv2.COLOR_RGBA2GRAY)
    gray=gray.reshape(-1)
    a.append(gray)

In [139]:
asdf=pd.DataFrame(a)
asdf

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
169,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
170,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
171,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [141]:
image_data = asdf.iloc[0].values
image_data.shape

(54400,)

### 이미지 증폭

In [142]:
# Albumentations에서 제공하는 여러 가지 증강 기법 중 일부 선택 (예시)

augmenter = Compose([
    HorizontalFlip(p=0.5),
    Rotate(limit=45, p=0.5),  # -45도에서 45도 사이의 임의의 각도로 회전
    GaussNoise(var_limit=(10.0, 50.0), p=0.5),  # 가우시안 노이즈 추가
    RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),  # 밝기와 대비 조절
])

augmented_rows = []

for row_index in range(len(asdf)):
    image_data = asdf.iloc[row_index].values
    image_data_reshaped = image_data.reshape(160, 340)

    # 원본 이미지 저장
    augmented_rows.append(image_data_reshaped.flatten())

    for _ in range(7):  # 원본 포함하여 5배로 증가시키기 위해 4번 반복
        # 증강된 이미지 생성 및 저장 
        augmented_image = augmenter(image=image_data_reshaped)['image']
        augmented_rows.append(augmented_image.flatten())

# DataFrame에 추가하기 위해 numpy array로 변환
augmented_array = np.array(augmented_rows)

# 새 DataFrame 생성 (원본 DataFrame의 컬럼 이름 사용)
df_augmented = pd.DataFrame(data=augmented_array, columns=asdf.columns)

### 이미지 라벨링

In [143]:
youngjaDF=df_augmented
youngjaDF

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,27,27,27,27,27,27,27,27,27,27,...,27,27,27,27,27,27,27,27,27,27
3,0,0,0,0,0,1,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1380,0,0,0,6,3,0,0,0,0,0,...,0,8,0,5,8,0,3,0,8,0
1381,10,3,1,8,4,0,2,0,0,0,...,0,2,0,3,0,4,3,0,0,0
1382,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [144]:
youngjaDF['woman_num']=4

## 옥순 데이터 전처리5

In [145]:
data = np.load('ouksoon.npz')
array6 = data['data1']
array6.shape

(210, 160, 340, 4)

### 4차원으로 배열변환

In [146]:
array6=array6.reshape(210,160, 340, 4)
array6.shape

(210, 160, 340, 4)

### 그레이스케일 후 이미지를 1차원으로 벡터로 변환

In [147]:
a=[]
for i in array6:
    gray = cv2.cvtColor(i, cv2.COLOR_RGBA2GRAY)
    gray=gray.reshape(-1)
    a.append(gray)

In [148]:
asdf=pd.DataFrame(a)
asdf

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,3,3,3,3,2,2,2,2,4,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
206,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
207,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
208,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [149]:
image_data = asdf.iloc[0].values
image_data.shape

(54400,)

### 이미지 증폭

In [150]:
# Albumentations에서 제공하는 여러 가지 증강 기법 중 일부 선택 (예시)

augmenter = Compose([
    HorizontalFlip(p=0.5),
    Rotate(limit=45, p=0.5),  # -45도에서 45도 사이의 임의의 각도로 회전
    GaussNoise(var_limit=(10.0, 50.0), p=0.5),  # 가우시안 노이즈 추가
    RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),  # 밝기와 대비 조절
])

augmented_rows = []

for row_index in range(len(asdf)):
    image_data = asdf.iloc[row_index].values
    image_data_reshaped = image_data.reshape(160, 340)

    # 원본 이미지 저장
    augmented_rows.append(image_data_reshaped.flatten())

    for _ in range(7):  # 원본 포함하여 5배로 증가시키기 위해 4번 반복
        # 증강된 이미지 생성 및 저장 
        augmented_image = augmenter(image=image_data_reshaped)['image']
        augmented_rows.append(augmented_image.flatten())

# DataFrame에 추가하기 위해 numpy array로 변환
augmented_array = np.array(augmented_rows)

# 새 DataFrame 생성 (원본 DataFrame의 컬럼 이름 사용)
df_augmented = pd.DataFrame(data=augmented_array, columns=asdf.columns)

In [151]:
ouksoonDF=df_augmented
ouksoonDF

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,2,9,0,0,3,0,0,0,6,...,7,0,5,4,5,6,4,0,0,4
2,54,50,50,56,51,50,50,50,51,50,...,50,54,50,53,52,52,54,50,50,50
3,65,61,69,61,52,73,65,51,62,63,...,22,32,26,21,21,21,21,21,25,23
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1675,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1676,27,26,28,34,27,25,25,27,25,25,...,37,27,27,25,25,25,25,25,25,25
1677,4,0,0,2,9,0,10,7,6,7,...,5,0,5,8,0,0,0,0,0,0
1678,15,6,7,12,18,6,7,10,12,6,...,9,6,6,10,13,13,12,6,6,6


### 이미지 라벨링

In [152]:
ouksoonDF['woman_num']=5

## 현숙 데이터 전처리6

In [153]:
data = np.load('hyunsook.npz')
array7 = data['data1']
array7.shape

(135, 160, 340, 4)

### 4차원으로 배열변환

In [154]:
array7=array7.reshape(135,160, 340, 4)
array7.shape

(135, 160, 340, 4)

### 그레이스케일 후 이미지를 1차원으로 벡터로 변환

In [155]:
a=[]
for i in array7:
    gray = cv2.cvtColor(i, cv2.COLOR_RGBA2GRAY)
    gray=gray.reshape(-1)
    a.append(gray)

In [156]:
asdf=pd.DataFrame(a)
asdf

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,237,234,234,235,241,245,248,246,235,215
2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,16,19,25,26,21,15,11,9,10,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
131,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,4
133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [157]:
image_data = asdf.iloc[0].values
image_data.shape

(54400,)

### 이미지 증폭

In [158]:
# Albumentations에서 제공하는 여러 가지 증강 기법 중 일부 선택 (예시)

augmenter = Compose([
    HorizontalFlip(p=0.5),
    Rotate(limit=45, p=0.5),  # -45도에서 45도 사이의 임의의 각도로 회전
    GaussNoise(var_limit=(10.0, 50.0), p=0.5),  # 가우시안 노이즈 추가
    RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),  # 밝기와 대비 조절
])

augmented_rows = []

for row_index in range(len(asdf)):
    image_data = asdf.iloc[row_index].values
    image_data_reshaped = image_data.reshape(160, 340)

    # 원본 이미지 저장
    augmented_rows.append(image_data_reshaped.flatten())

    for _ in range(7):  # 원본 포함하여 5배로 증가시키기 위해 4번 반복
        # 증강된 이미지 생성 및 저장 
        augmented_image = augmenter(image=image_data_reshaped)['image']
        augmented_rows.append(augmented_image.flatten())

# DataFrame에 추가하기 위해 numpy array로 변환
augmented_array = np.array(augmented_rows)

# 새 DataFrame 생성 (원본 DataFrame의 컬럼 이름 사용)
df_augmented = pd.DataFrame(data=augmented_array, columns=asdf.columns)

In [159]:
hyunsookDF=df_augmented
hyunsookDF

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,4,5,13,8,0,14,0,0,0,...,0,6,0,0,2,0,7,0,4,5
2,0,0,1,4,3,2,0,0,0,0,...,0,0,0,0,0,1,0,0,7,3
3,27,27,27,27,27,27,27,27,27,27,...,27,27,27,27,27,27,27,27,27,27
4,0,0,3,0,0,5,6,0,12,0,...,0,0,10,6,2,7,4,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,0,0,0,2,1,4,4,2,5,2,...,0,6,0,4,2,4,2,2,1,0
1076,0,6,3,0,0,0,0,0,0,11,...,6,0,2,0,0,1,0,0,0,0
1077,16,16,16,16,16,16,16,16,16,16,...,16,16,16,16,16,16,16,16,16,16
1078,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 이미지 라벨링

In [160]:
hyunsookDF['woman_num']=6

### 데이터 병합 후 넘파이 파일로 저장

In [161]:
womanDF=pd.concat([youngsookDF, jungsookDF])
womanDF=pd.concat([womanDF, soonjaDF])
womanDF=pd.concat([womanDF, youngjaDF])
womanDF=pd.concat([womanDF, ouksoonDF])
womanDF=pd.concat([womanDF, hyunsookDF])
womanDF

,0,1,2,3,4,5,6,7,8,9,...,54391,54392,54393,54394,54395,54396,54397,54398,54399,woman_num
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,1,0,2,0,0,0,...,0,3,0,1,0,1,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,4,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,0,0,0,2,1,4,4,2,5,2,...,6,0,4,2,4,2,2,1,0,6
1076,0,6,3,0,0,0,0,0,0,11,...,0,2,0,0,1,0,0,0,0,6
1077,16,16,16,16,16,16,16,16,16,16,...,16,16,16,16,16,16,16,16,16,6
1078,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6


In [162]:
np.save('womanDF.npy', womanDF)